Preprocessing Anime dataset from Kaggle

In [3]:
from collections import defaultdict
import pandas as pd
import numpy as np
import sklearn as sk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel
import matplotlib.pyplot as plt
import re
import string

1. Data preprocessing

1.1 Data Cleaning

In [4]:
# remove unwanted features (columns) from the dataset
anime_df = pd.read_csv("animes.csv")
anime_df.rename(columns={'title': 'name'}, inplace=True)
anime_df.drop(['aired', 'ranked', 'img_url', 'link'], axis=1, inplace=True)

#removing unwanted characters from the anime name strings
def text_cleaning(text):
    text = re.sub(r'&quot;', '', text)
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub(r'.hack//', '', text)
    text = re.sub(r'&#039;', '', text)
    text = re.sub(r'A&#039;s', '', text)
    text = re.sub(r'I&#039;', 'I\'', text)
    text = re.sub(r'&amp;', 'and', text)
    text = re.sub(r'Â°', '',text)

    return text

anime_df['name'] = anime_df['name'].apply(text_cleaning)
anime_df.head(5)

,uid,name,synopsis,genre,episodes,members,popularity,score
0,28891,Haikyuu Second Season,Following their participation at the Inter-Hig...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...",25.0,489888,141,8.82
1,23273,Shigatsu wa Kimi no Uso,Music accompanies the path of the human metron...,"['Drama', 'Music', 'Romance', 'School', 'Shoun...",22.0,995473,28,8.83
2,34599,Made in Abyss,The Abyss—a gaping chasm stretching down into ...,"['Sci-Fi', 'Adventure', 'Mystery', 'Drama', 'F...",13.0,581663,98,8.83
3,5114,Fullmetal Alchemist Brotherhood,"""In order for something to be obtained, someth...","['Action', 'Military', 'Adventure', 'Comedy', ...",64.0,1615084,4,9.23
4,31758,Kizumonogatari III Reiketsuhen,After helping revive the legendary vampire Kis...,"['Action', 'Mystery', 'Supernatural', 'Vampire']",1.0,214621,502,8.83


In [5]:
anime_df.rename(columns={'uid': 'anime_uid', 'score': 'rating'}, inplace=True)
anime_df.episodes.replace({'Unknown':np.nan},inplace=True)

anime_df.drop_duplicates(subset=['name'], inplace=True)
anime_df.dropna(inplace=True)
anime_df.reset_index(drop=True, inplace=True)

anime_df.head(5)

,anime_uid,name,synopsis,genre,episodes,members,popularity,rating
0,28891,Haikyuu Second Season,Following their participation at the Inter-Hig...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...",25.0,489888,141,8.82
1,23273,Shigatsu wa Kimi no Uso,Music accompanies the path of the human metron...,"['Drama', 'Music', 'Romance', 'School', 'Shoun...",22.0,995473,28,8.83
2,34599,Made in Abyss,The Abyss—a gaping chasm stretching down into ...,"['Sci-Fi', 'Adventure', 'Mystery', 'Drama', 'F...",13.0,581663,98,8.83
3,5114,Fullmetal Alchemist Brotherhood,"""In order for something to be obtained, someth...","['Action', 'Military', 'Adventure', 'Comedy', ...",64.0,1615084,4,9.23
4,31758,Kizumonogatari III Reiketsuhen,After helping revive the legendary vampire Kis...,"['Action', 'Mystery', 'Supernatural', 'Vampire']",1.0,214621,502,8.83


In [6]:
# replace the characters "[]'" with an empty space as the genre column is already of type string
anime_df['genre'] = anime_df['genre'].str.replace("'", "", regex=False)
anime_df['genre'] = anime_df['genre'].str.replace("[", "", regex=False)
anime_df['genre'] = anime_df['genre'].str.replace("]", "", regex=False)

anime_df.head(5)

,anime_uid,name,synopsis,genre,episodes,members,popularity,rating
0,28891,Haikyuu Second Season,Following their participation at the Inter-Hig...,"Comedy, Sports, Drama, School, Shounen",25.0,489888,141,8.82
1,23273,Shigatsu wa Kimi no Uso,Music accompanies the path of the human metron...,"Drama, Music, Romance, School, Shounen",22.0,995473,28,8.83
2,34599,Made in Abyss,The Abyss—a gaping chasm stretching down into ...,"Sci-Fi, Adventure, Mystery, Drama, Fantasy",13.0,581663,98,8.83
3,5114,Fullmetal Alchemist Brotherhood,"""In order for something to be obtained, someth...","Action, Military, Adventure, Comedy, Drama, Ma...",64.0,1615084,4,9.23
4,31758,Kizumonogatari III Reiketsuhen,After helping revive the legendary vampire Kis...,"Action, Mystery, Supernatural, Vampire",1.0,214621,502,8.83


In [7]:
normalised_anime_df = anime_df.copy()

weights = {
    'genre': 0.35,
    'members_norm': 0.1,
    'rating_norm': 0.35,
    'popularity_norm': 0.1,
    'episodes_norm': 0.1
}

normalised_anime_df['members_norm'] = normalised_anime_df['members'] / normalised_anime_df['members'].max() * weights['members_norm']
normalised_anime_df['rating_norm'] = normalised_anime_df['rating'] / normalised_anime_df['rating'].max() * weights['rating_norm']
normalised_anime_df['popularity_norm'] = normalised_anime_df['popularity'] / normalised_anime_df['popularity'].max() * weights['popularity_norm']
normalised_anime_df['episodes_norm'] = normalised_anime_df['episodes'] / normalised_anime_df['episodes'].max() * weights['episodes_norm']

normalised_anime_df.head()

,anime_uid,name,synopsis,genre,episodes,members,popularity,rating,members_norm,rating_norm,popularity_norm,episodes_norm
0,28891,Haikyuu Second Season,Following their participation at the Inter-Hig...,"Comedy, Sports, Drama, School, Shounen",25.0,489888,141,8.82,0.026183,0.334453,0.000864,0.000818
1,23273,Shigatsu wa Kimi no Uso,Music accompanies the path of the human metron...,"Drama, Music, Romance, School, Shounen",22.0,995473,28,8.83,0.053204,0.334832,0.000172,0.000720
2,34599,Made in Abyss,The Abyss—a gaping chasm stretching down into ...,"Sci-Fi, Adventure, Mystery, Drama, Fantasy",13.0,581663,98,8.83,0.031088,0.334832,0.000600,0.000425
3,5114,Fullmetal Alchemist Brotherhood,"""In order for something to be obtained, someth...","Action, Military, Adventure, Comedy, Drama, Ma...",64.0,1615084,4,9.23,0.086320,0.350000,0.000025,0.002094
4,31758,Kizumonogatari III Reiketsuhen,After helping revive the legendary vampire Kis...,"Action, Mystery, Supernatural, Vampire",1.0,214621,502,8.83,0.011471,0.334832,0.003076,0.000033


In [8]:
normalised_anime_df.drop(['members', 'rating', 'popularity', 'episodes'], axis=1, inplace=True)

normalised_anime_df.head()

,anime_uid,name,synopsis,genre,members_norm,rating_norm,popularity_norm,episodes_norm
0,28891,Haikyuu Second Season,Following their participation at the Inter-Hig...,"Comedy, Sports, Drama, School, Shounen",0.026183,0.334453,0.000864,0.000818
1,23273,Shigatsu wa Kimi no Uso,Music accompanies the path of the human metron...,"Drama, Music, Romance, School, Shounen",0.053204,0.334832,0.000172,0.000720
2,34599,Made in Abyss,The Abyss—a gaping chasm stretching down into ...,"Sci-Fi, Adventure, Mystery, Drama, Fantasy",0.031088,0.334832,0.000600,0.000425
3,5114,Fullmetal Alchemist Brotherhood,"""In order for something to be obtained, someth...","Action, Military, Adventure, Comedy, Drama, Ma...",0.086320,0.350000,0.000025,0.002094
4,31758,Kizumonogatari III Reiketsuhen,After helping revive the legendary vampire Kis...,"Action, Mystery, Supernatural, Vampire",0.011471,0.334832,0.003076,0.000033


In [9]:
genres_df = anime_df['genre'].str.get_dummies(sep=', ').astype(int)
genres_df = genres_df.apply(lambda x : x * weights['genre'])

genres_df.head()

,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,Fantasy,Game,...,Shounen Ai,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Vampire,Yaoi,Yuri
0,0.00,0.00,0.0,0.35,0.0,0.0,0.35,0.0,0.00,0.0,...,0.0,0.0,0.0,0.35,0.0,0.00,0.0,0.00,0.0,0.0
1,0.00,0.00,0.0,0.00,0.0,0.0,0.35,0.0,0.00,0.0,...,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0
2,0.00,0.35,0.0,0.00,0.0,0.0,0.35,0.0,0.35,0.0,...,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0
3,0.35,0.35,0.0,0.35,0.0,0.0,0.35,0.0,0.35,0.0,...,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0
4,0.35,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,...,0.0,0.0,0.0,0.00,0.0,0.35,0.0,0.35,0.0,0.0


In [10]:
normalised_anime_df.drop('genre', axis=1, inplace=True)
normalised_anime_df = pd.concat([normalised_anime_df, genres_df], axis=1)

normalised_anime_df.head()

,anime_uid,name,synopsis,members_norm,rating_norm,popularity_norm,episodes_norm,Action,Adventure,Cars,...,Shounen Ai,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Vampire,Yaoi,Yuri
0,28891,Haikyuu Second Season,Following their participation at the Inter-Hig...,0.026183,0.334453,0.000864,0.000818,0.00,0.00,0.0,...,0.0,0.0,0.0,0.35,0.0,0.00,0.0,0.00,0.0,0.0
1,23273,Shigatsu wa Kimi no Uso,Music accompanies the path of the human metron...,0.053204,0.334832,0.000172,0.000720,0.00,0.00,0.0,...,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0
2,34599,Made in Abyss,The Abyss—a gaping chasm stretching down into ...,0.031088,0.334832,0.000600,0.000425,0.00,0.35,0.0,...,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0
3,5114,Fullmetal Alchemist Brotherhood,"""In order for something to be obtained, someth...",0.086320,0.350000,0.000025,0.002094,0.35,0.35,0.0,...,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0
4,31758,Kizumonogatari III Reiketsuhen,After helping revive the legendary vampire Kis...,0.011471,0.334832,0.003076,0.000033,0.35,0.00,0.0,...,0.0,0.0,0.0,0.00,0.0,0.35,0.0,0.35,0.0,0.0


In [11]:
features = ['members_norm', 'rating_norm', 'popularity_norm', 'episodes_norm'] + genres_df.columns.tolist()

cosine_sim = cosine_similarity(normalised_anime_df[features], normalised_anime_df[features])

print(cosine_sim)

[[1.         0.66214591 0.32441239 ... 0.38178832 0.24895614 0.65314481]
 [0.66214591 1.         0.32520761 ... 0.16702495 0.24895703 0.65247583]
 [0.32441239 0.32520761 1.         ... 0.16711943 0.24916537 0.30803338]
 ...
 [0.38178832 0.16702495 0.16711943 ... 1.         0.26943555 0.58509904]
 [0.24895614 0.24895703 0.24916537 ... 0.26943555 1.         0.21806531]
 [0.65314481 0.65247583 0.30803338 ... 0.58509904 0.21806531 1.        ]]


In [12]:
indices = pd.Series(anime_df.index, index=anime_df['name']).drop_duplicates()

Content based filtering recommendations

In [36]:
def content_based_recommendations(title, cosine_sim=cosine_sim, anime_df=anime_df, indices=indices):
    # Get the index of the anime that matches the title
    idx = indices[title]

    # Get the pairwise cosine similarity scores for all anime with that index
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the anime based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the top 10 most similar anime
    # sim_scores = sim_scores[1:11]

    # Get the titles of the top 10 most similar anime
    # anime_indices = [i[0] for i in sim_scores]
    # anime_titles = anime_df['name'].iloc[anime_indices].values.tolist()


    # the aim here is to stop different seaseons of an anime being recommended. 
    # E.g. Tokyo Ghoul Season 1, Tokyo Ghoul Season 2, etc.
    # STILL INCOMPLETE -> perhaps try to fix in the item-item recommendations (chatgpt)

    anime_indices = []

    for i in sim_scores:
        name = anime_df['name'].iloc[i[0]]
        if len(anime_indices) == 10: # providing 10 recommendations
            break
        if re.search(title, name):
            continue

        anime_indices.append(i[0])

    anime_titles = anime_df['name'].iloc[anime_indices].values.tolist()

    return anime_titles

In [37]:
content_based_recommendations('Death Note')

['Mirai Nikki',
 'B The Beginning',
 'Higurashi no Naku Koro ni Kai',
 'Mousou Dairinin',
 'Higurashi no Naku Koro ni Rei',
 'Imawa no Kuni no Alice OVA',
 'Mouryou no Hako',
 'Zankyou no Terror',
 'Jigoku Shoujo Mitsuganae',
 'Babylon']

Item-item Collaborative filtering

From the recommendations provided from the content based algorithm above, calculate the similarity of each of those recommendations with anime the user has already watched.

In [15]:
user_ratings_df = pd.read_csv("reviews.csv")
user_ratings_df.drop(['link', 'text'], axis=1, inplace=True)
user_ratings_df.rename(columns={'profile': 'user_id'}, inplace=True)

user_ratings_df.head()

,uid,user_id,anime_uid,score,scores
0,255938,DesolatePsyche,34096,8,"{'Overall': '8', 'Story': '8', 'Animation': '8..."
1,259117,baekbeans,34599,10,"{'Overall': '10', 'Story': '10', 'Animation': ..."
2,253664,skrn,28891,7,"{'Overall': '7', 'Story': '7', 'Animation': '9..."
3,8254,edgewalker00,2904,9,"{'Overall': '9', 'Story': '9', 'Animation': '9..."
4,291149,aManOfCulture99,4181,10,"{'Overall': '10', 'Story': '10', 'Animation': ..."


In [16]:
# # change profile names into unique IDs (i.e. integers)
user_ratings_df.user_id = pd.factorize(user_ratings_df.user_id)[0]

user_ratings_df.head(10)

,uid,user_id,anime_uid,score,scores
0,255938,0,34096,8,"{'Overall': '8', 'Story': '8', 'Animation': '8..."
1,259117,1,34599,10,"{'Overall': '10', 'Story': '10', 'Animation': ..."
2,253664,2,28891,7,"{'Overall': '7', 'Story': '7', 'Animation': '9..."
3,8254,3,2904,9,"{'Overall': '9', 'Story': '9', 'Animation': '9..."
4,291149,4,4181,10,"{'Overall': '10', 'Story': '10', 'Animation': ..."
5,10046,5,2904,10,"{'Overall': '10', 'Story': '9', 'Animation': '..."
6,247454,6,16664,6,"{'Overall': '6', 'Story': '6', 'Animation': '9..."
7,140903,7,2904,8,"{'Overall': '8', 'Story': '7', 'Animation': '9..."
8,23791,8,2904,10,"{'Overall': '10', 'Story': '10', 'Animation': ..."
9,25115,9,4181,4,"{'Overall': '4', 'Story': '8', 'Animation': '9..."


In [17]:
user_ratings_df['user_id'].value_counts()

2762     762
456      743
109      648
97       566
31       491
        ... 
36472      1
36473      1
36476      1
36481      1
47884      1
Name: user_id, Length: 47885, dtype: int64

In [18]:
anime_with_ratings_df = pd.merge(anime_df, user_ratings_df, on='anime_uid')

anime_with_ratings_df.drop_duplicates(subset=['user_id', 'name'], inplace=True)
anime_with_ratings_df.reset_index(drop=True, inplace=True)

anime_with_ratings_df.head()

,anime_uid,name,synopsis,genre,episodes,members,popularity,rating,uid,user_id,score,scores
0,28891,Haikyuu Second Season,Following their participation at the Inter-Hig...,"Comedy, Sports, Drama, School, Shounen",25.0,489888,141,8.82,253664,2,7,"{'Overall': '7', 'Story': '7', 'Animation': '9..."
1,28891,Haikyuu Second Season,Following their participation at the Inter-Hig...,"Comedy, Sports, Drama, School, Shounen",25.0,489888,141,8.82,254554,15102,10,"{'Overall': '10', 'Story': '10', 'Animation': ..."
2,28891,Haikyuu Second Season,Following their participation at the Inter-Hig...,"Comedy, Sports, Drama, School, Shounen",25.0,489888,141,8.82,271227,15103,10,"{'Overall': '10', 'Story': '10', 'Animation': ..."
3,28891,Haikyuu Second Season,Following their participation at the Inter-Hig...,"Comedy, Sports, Drama, School, Shounen",25.0,489888,141,8.82,284956,13930,7,"{'Overall': '7', 'Story': '7', 'Animation': '7..."
4,28891,Haikyuu Second Season,Following their participation at the Inter-Hig...,"Comedy, Sports, Drama, School, Shounen",25.0,489888,141,8.82,249279,599,8,"{'Overall': '8', 'Story': '7', 'Animation': '7..."


Create User-item matrix for item-item collaborative filtering

In [19]:
anime_rating_pivot = anime_with_ratings_df.pivot_table(index='user_id', columns='name', values='score') #maybe use the mean score value to fill in NaN instead of 0s?, fill_value=7
anime_rating_pivot.fillna(0, inplace=True)

anime_rating_pivot.head(10)

name,0,0008,001,009 ReCyborg,0091,07Ghost,100,100 Pascalsensei TV,1000nen Joou Queen Millennia,1001 Nights,...,makemagic,sCRYed,vivi,xxxHOLiC,xxxHOLiC Kei,xxxHOLiC Movie Manatsu no Yoru no Yume,xxxHOLiC Rou,xxxHOLiC Shunmuki,ēlDLIVE,◯
user_id,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
item_similarities = cosine_similarity(anime_rating_pivot.T)
item_similarities_df = pd.DataFrame(item_similarities, index=anime_rating_pivot.columns, columns=anime_rating_pivot.columns)

item_similarities_df.head(10)

name,0,0008,001,009 ReCyborg,0091,07Ghost,100,100 Pascalsensei TV,1000nen Joou Queen Millennia,1001 Nights,...,makemagic,sCRYed,vivi,xxxHOLiC,xxxHOLiC Kei,xxxHOLiC Movie Manatsu no Yoru no Yume,xxxHOLiC Rou,xxxHOLiC Shunmuki,ēlDLIVE,◯
name,,,,,,,,,,,,,,,,,,,,,
0,1.000000,0.261557,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
0008,0.261557,1.000000,0.026801,0.083551,0.052803,0.000000,0.0,0.0,0.0,0.027276,...,0.0,0.0,0.161796,0.0,0.0,0.0,0.0,0.0,0.0,0.053602
001,0.000000,0.026801,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.027925,...,0.0,0.0,0.165647,0.0,0.0,0.0,0.0,0.0,0.0,0.164634
009 ReCyborg,0.000000,0.083551,0.000000,1.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
0091,0.000000,0.052803,0.000000,0.000000,1.000000,0.084706,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
07Ghost,0.000000,0.000000,0.000000,0.000000,0.084706,1.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
100,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
100 Pascalsensei TV,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,1.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1000nen Joou Queen Millennia,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,1.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [38]:
def collaborative_filtering_recommendations(anime, n=10):
    similarity_scores = item_similarities_df[anime]
    similarity_scores = similarity_scores.sort_values(ascending=False)

    similar_anime = similarity_scores.iloc[1:n+1].index.tolist()

    return similar_anime


In [39]:
collaborative_filtering_recommendations('Death Note')

['Code Geass Hangyaku no Lelouch',
 'Fullmetal Alchemist Brotherhood',
 'SteinsGate',
 'Code Geass Hangyaku no Lelouch R2',
 'Elfen Lied',
 'Sword Art Online',
 'Cowboy Bebop',
 'Monster',
 'Deadman Wonderland',
 'Tokyo Ghoul']

Hybrid Implementation: Combine content based and collaborative filtering methods to provide recommendations to the user

calculate weighted similarity score for each anime:

get scores for each anime from each filterng method and multiply it accordingly (0.5 for cf, 0.5 for cbf)

In [40]:
# content_based_anime_indices = indices.copy()
# content_based_sim_scores = cosine_sim.copy()

collaborative_sim_scores = item_similarities_df.copy()

In [52]:
def combined_recommendations(anime_name, num_recommendations=10, content_weight=0.5):
    content_based = content_based_recommendations(anime_name)
    collaborative_filtering = collaborative_filtering_recommendations(anime_name)

    recommendations = list(set(content_based + collaborative_filtering))
    similar_anime_subset = collaborative_sim_scores.loc[recommendations]
    # print(similar_anime_subset)
    # similar_anime_subset.fillna(similar_anime_subset.mean(), inplace=True)

    content_based_scores = collaborative_sim_scores.loc[content_based]
    collaborative_filtering_scores = collaborative_sim_scores.loc[collaborative_filtering]
    print('CONTENT: ', content_based_scores)
    print('COLLAB: ', collaborative_filtering_scores)
    
    scores = content_weight * content_based_scores + (1 - content_weight) * collaborative_filtering_scores
    # print(scores)
    # scores = scores.sort_values(by=, ascending=False)

    return scores.head(num_recommendations).index.tolist()

In [53]:
combined_recommendations('Death Note')

name                                      0      0008       001  009 ReCyborg  \
name                                                                            
Sword Art Online                   0.000000  0.003141  0.000000      0.000000   
Elfen Lied                         0.000000  0.000000  0.000000      0.006134   
Tokyo Ghoul                        0.000000  0.000000  0.000000      0.000000   
Higurashi no Naku Koro ni Kai      0.000000  0.000000  0.000000      0.000000   
Higurashi no Naku Koro ni Rei      0.000000  0.000000  0.000000      0.000000   
Deadman Wonderland                 0.036472  0.030662  0.000000      0.033287   
Fullmetal Alchemist Brotherhood    0.000000  0.000000  0.000000      0.000000   
Monster                            0.000000  0.000000  0.000000      0.015595   
Babylon                            0.000000  0.000000  0.000000      0.000000   
Code Geass Hangyaku no Lelouch     0.000000  0.000000  0.000000      0.000000   
Zankyou no Terror           

['B The Beginning',
 'Babylon',
 'Code Geass Hangyaku no Lelouch',
 'Code Geass Hangyaku no Lelouch R2',
 'Cowboy Bebop',
 'Deadman Wonderland',
 'Elfen Lied',
 'Fullmetal Alchemist Brotherhood',
 'Higurashi no Naku Koro ni Kai',
 'Higurashi no Naku Koro ni Rei']

In [24]:
# merged_anime_reviews_df = pd.merge(anime_df, user_ratings_df, on='anime_uid')
# merged_anime_reviews_df.rename(columns={'score_x':'avg_rating', 'text': 'review_text'},inplace=True)

# merged_anime_reviews_df.drop('review_text', axis=1, inplace=True)

# merged_anime_reviews_df.head()

Categorical encoding -> Separating genres into their own respective column

In [25]:
# replace the characters "[]'" with an empty space as the genre column is already of type string
# merged_anime_reviews_df['genre'] = merged_anime_reviews_df['genre'].str.replace("'", "", regex=False)
# merged_anime_reviews_df['genre'] = merged_anime_reviews_df['genre'].str.replace("[", "", regex=False)
# merged_anime_reviews_df['genre'] = merged_anime_reviews_df['genre'].str.replace("]", "", regex=False)

# merged_anime_reviews_df.head(5)